## Add New PC_Overview Rows to All Stack

In [1]:
import os
import pandas as pd
import openpyxl
from openpyxl.styles import Font
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.table import Table, TableStyleInfo
import re

In [2]:
def filter_sheets(input_file_path, output_file_path, po_numbers_to_exclude):
    # Read the Excel file
    with pd.ExcelFile(input_file_path) as xls:
        df_stack = pd.read_excel(xls, "Stack")
        df_pc_overview = pd.read_excel(xls, "pc_overview")

    # Filter out rows with the specified PO numbers and keep the removed rows
    df_stack_filtered = df_stack[~df_stack["PO #"].isin(po_numbers_to_exclude)]
    df_stack_removed = df_stack[df_stack["PO #"].isin(po_numbers_to_exclude)]

    df_pc_overview_filtered = df_pc_overview[~df_pc_overview["PO #"].isin(po_numbers_to_exclude)]
    df_pc_overview_removed = df_pc_overview[df_pc_overview["PO #"].isin(po_numbers_to_exclude)]

    # Save the filtered data and removed rows to a new Excel file
    with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
        df_stack_filtered.to_excel(writer, sheet_name="Stack", index=False)
        df_pc_overview_filtered.to_excel(writer, sheet_name="pc_overview", index=False)
        df_stack_removed.to_excel(writer, sheet_name="Stack_Removed", index=False)
        df_pc_overview_removed.to_excel(writer, sheet_name="pc_overview_Removed", index=False)

    print(f"Filtered data and removed rows saved to {output_file_path}")
    print(f"Rows removed from Stack: {len(df_stack_removed)}")
    print(f"Rows removed from pc_overview: {len(df_pc_overview_removed)}")

# Updated PO numbers to exclude
po_numbers_to_exclude = [
    "USCSB2307079A", "USCSB2308094A", "USCSB2109007A", "USCSB2306043A", "USCSB2205003A",
    "USCSB2306078A", "USCSB2306029A", "USCSB2403154A", "USCSB2305094A", "USC2301008A",
    "USCSB2306077A", "USCSB2307120A", "USCSB2307121A", "USCSB2205007A", "USCSB2301045A",
    "USCSB2406081A", "USCSB2405066A", "USCSB2405061A", "USCSB2308142A", "USCSB2308173A",
    "USCSB2301026A", "USCSB2208007A", "USCSB2306120A", "USCSB2309112A", "USCSB2306130A",
    "USCSB2404182A", "USCSB2408133A", "USCSB2307113A", "USCSB2405142A", "USCSB2308177A",
    "USCSB2305079A", "USCSB2205026A", "USCSB2404156A", "USCSB2404097A", "USCSB2205016A",
    "USCSB2405089A", "USCSB2312149A", "USCSB2302056A", "USCSB2306102A", "USCSB2406056A",
    "USCSB2403037A", "USCSB2404139A", "USCSB2307024A", "USCSB2404040A", "USCSB2310090A",
    "USCSB2306049A", "USCSB2402195A", "USCSB2402086A", "USCSB2308180A", "USCSB2402151A",
    "USCSB2405032A", "USCSB2309010A", "USCSB2310055A", "USCSB2308050A", "USCSB2310139A",
    "USCSB2402210A", "USCSB2403180A", "USCSB2407146A", "USCSB2405102A", "USCSB2310170A",
    "USC2111048A", "USC2309038B", "USCSB2310116A", "USCSB2212045A", "USCSB2308004A",
    "USCSB2407117A", "USCSB2309026A", "USCSB2404022A", "USC2402072A", "USCSB2308070A",
    "USCSB2407090A", "USCSB2406116A", "USCSB2401044A"
]

# File paths using os.path.join() for cross-platform compatibility
input_file_path = os.path.join('AP_Files', 'All_Stack.xlsx')
output_file_path = os.path.join('AP_Files', 'All_Stack_Filtered.xlsx')


# Run the function
filter_sheets(input_file_path, output_file_path, po_numbers_to_exclude)

Filtered data and removed rows saved to AP_Files/All_Stack_Filtered.xlsx
Rows removed from Stack: 73
Rows removed from pc_overview: 73


In [3]:
# File paths
ALL_STACK_PATH = os.path.join("AP_Files", "All_Stack_Filtered.xlsx")
OUTPUT_PATH = os.path.join("AP_Files", "updated_all_stack.xlsx")



# Helper functions
def auto_adjust_column_width(worksheet):
    for column in worksheet.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2) * 1.2
        worksheet.column_dimensions[column_letter].width = adjusted_width

def format_as_table(worksheet, data_range, style='TableStyleMedium9'):
    table_name = re.sub(r'\W+', '', worksheet.title)
    table_name = f"Table_{table_name}"
    
    table = Table(displayName=table_name, ref=data_range)
    style = TableStyleInfo(name=style, showFirstColumn=False,
                           showLastColumn=False, showRowStripes=True, showColumnStripes=False)
    table.tableStyleInfo = style
    
    worksheet.add_table(table)

def add_total_row(worksheet):
    last_row = worksheet.max_row
    last_col = worksheet.max_column
    
    amount_col = None
    for col in range(1, last_col + 1):
        if worksheet.cell(row=1, column=col).value == 'Sum of Amount':
            amount_col = col
            break
    
    if amount_col is not None:
        worksheet.cell(row=last_row + 1, column=1, value='Total')
        sum_formula = f'=SUM({get_column_letter(amount_col)}2:{get_column_letter(amount_col)}{last_row})'
        worksheet.cell(row=last_row + 1, column=amount_col, value=sum_formula)
        
        for col in range(1, last_col + 1):
            worksheet.cell(row=last_row + 1, column=col).font = Font(bold=True)

def process_data():
    # Read data from All Stack Analysis
    pc_overview = pd.read_excel(ALL_STACK_PATH, sheet_name="pc_overview")
    stack_data = pd.read_excel(ALL_STACK_PATH, sheet_name="Stack")
    
    # Identify new rows in pc_overview that are not in stack_data
    new_rows = pc_overview[~pc_overview['PO #'].isin(stack_data['PO #'])]

    # Prepare new rows for Stack
    new_stack_rows = new_rows[['Project Number', 'PO #', 'PO Description', 'Vendor/Subcontractor', 'Amount']].copy()
    # new_stack_rows = new_stack_rows.rename(columns={
    #     'Amount': 'Sum of Amount'
    # })

    # Append new rows to stack_data
    stack_data_updated = pd.concat([stack_data, new_stack_rows], ignore_index=True)

    return stack_data_updated

def create_summary_sheets(stack_data, writer):
    # Write the updated Stack sheet as pc_overview in the summary file
    stack_data.to_excel(writer, sheet_name='pc_overview', index=False)
    worksheet = writer.sheets['pc_overview']
    auto_adjust_column_width(worksheet)
    format_as_table(worksheet, f"A1:{get_column_letter(worksheet.max_column)}{worksheet.max_row}")
    # add_total_row(worksheet)

# Main execution
if __name__ == "__main__":
    # Process data
    updated_stack_data = process_data()

    # Create summary sheets
    with pd.ExcelWriter(OUTPUT_PATH, engine='openpyxl') as writer:
        create_summary_sheets(updated_stack_data, writer)

    print(f"Summary tables have been updated and saved to {OUTPUT_PATH}")

Summary tables have been updated and saved to AP_Files/updated_all_stack.xlsx


## Add Manually Updated Stack Sheet to All Stack

In [4]:
# File paths
ALL_STACK_PATH = os.path.join("AP_Files", "All_Stack.xlsx")
OUTPUT_PATH = os.path.join("AP_Files", "updated_all_stack.xlsx")

def update_stack_sheet():
    # Read both files
    summary_data = pd.read_excel(OUTPUT_PATH, sheet_name='pc_overview')
    stack_data = pd.read_excel(ALL_STACK_PATH, sheet_name='Stack')
    
    # Find rows in summary_data where PO # is not in stack_data
    new_rows = summary_data[~summary_data['PO #'].isin(stack_data['PO #'])]
    
    # Show information and get confirmation
    print(f"\nFound {len(new_rows)} new rows to add to Stack sheet.")
    if len(new_rows) > 0:
        sample_po = new_rows['PO #'].iloc[0]
        print(f"First new PO #: {sample_po}")
        
        confirm = input("\nDo you want to proceed with updating the Stack sheet? (yes/no): ").lower()
        
        if confirm != 'yes':
            print("Update cancelled.")
            return
        
        # Proceed with update if confirmed
        with pd.ExcelWriter(ALL_STACK_PATH, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            # Append new rows to stack_data
            updated_stack = pd.concat([stack_data, new_rows], ignore_index=True)
            
            # Write back to Stack sheet
            updated_stack.to_excel(writer, sheet_name='Stack', index=False)
            
            print(f"\nSuccessfully added {len(new_rows)} new rows to Stack sheet")
    else:
        print("No new rows found to add.")

if __name__ == "__main__":
    print("Starting Stack sheet update process...")
    update_stack_sheet()


Starting Stack sheet update process...

Found 102 new rows to add to Stack sheet.
First new PO #: US2SB2410003A
Update cancelled.
